In [1]:
from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
from ipsl_dcpp.model.pangu import PanguWeather
from hydra import compose, initialize
from omegaconf import OmegaConf
import pickle
import hydra
import os
import numpy as np
import pandas as pd

with initialize(version_base=None, config_path="ipsl_dcpp/conf"):
    cfg = compose(config_name="config")
pl.seed_everything(cfg.experiment.seed)

val = hydra.utils.instantiate(
    cfg.experiment.val_dataset,
    generate_statistics=False,
    surface_variables=cfg.experiment.surface_variables,
    depth_variables=cfg.experiment.depth_variables,
    plev_variables=cfg.experiment.plev_variables,
    work_path=cfg.environment.work_path,
    scratch_path=cfg.environment.scratch_path,
)

val_dataloader = torch.utils.data.DataLoader(
    val,
    batch_size=1,
    shuffle=False,
    num_workers=1
)

#batch = next(iter(train_dataloader))
model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(
        cfg.experiment.backbone,
    ),
    dataset=val_dataloader.dataset

)



Seed set to 0
9it [00:02,  3.70it/s]
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025535429/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:

# trainer.logged_metrics
x = np.stack(val.timestamps)[:,2]
indices = np.stack(val.timestamps)[np.where((pd.to_datetime(x).year == 2001) & (pd.to_datetime(x).month == 1),True,False)][:,[0,1]]

checkpoint_path = torch.load(f'ipsl_dcpp/epoch=16.ckpt',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint_path['state_dict'])
# trainer.test(model, val_dataloader)
inv_map = {v: k for k, v in val.id2pt.items()}


for i in range(1):
    index = inv_map[(indices[i][0],indices[i][1])]
    batch = val.__getitem__(index)
    batch['state_constant'] = torch.Tensor(batch['state_constant'])
    batch['time'] = [batch['time']]
    batch['next_time'] = [batch['next_time']]
    batch = {k: v.unsqueeze(0) if (k != 'time') and (k != 'next_time') else v for k, v in batch.items()}
    output = model.sample_rollout(batch, lead_time_months=1)
    #with open(f'{i}_rollout_from_diff_ensemble_with_t_and_prev.pkl','wb') as f:
    #     pickle.dump(output,f)

tensor(900)
tensor(800)


RuntimeError: Given groups=1, weight of size [192, 28, 2, 2], expected input[1, 47, 144, 144] to have 28 channels, but got 47 channels instead